In [ ]:
import os

import geopandas as gpd
import pandas as pd
import shapely.geometry as sg

from aerial_image_detection.raster_utils import RasterData
from aerial_image_detection.sahi_inference import SAHIInferenceModel

RD_crs = "EPSG:28992"

classes_to_keep = [9, 10]  # ["large vehicle", "small vehicle"]

full_images_folder = "../datasets/experiments/parkeren/luchtfotos/luchtfotos_centrum_2025/"
output_folder = "../datasets/experiments/parkeren/detections/"

In [ ]:
sahi_model = SAHIInferenceModel(
    yolo_model_weights_path="../datasets/experiments/parkeren/model_weights/yolo11m-obb.pt",
    confidenc_treshold=0.01,
    image_size=1024,
    slice_height=1024,
    slice_width=1024,
    classes_to_keep=classes_to_keep,
    class_agnostic=True
)

In [ ]:
images = sorted([file for file in os.listdir(full_images_folder) if file.endswith(".tif")])

os.makedirs(output_folder, exist_ok=True)

detections_gdf_list = []

In [ ]:
for i, image_file in enumerate(images):
    print(f"Processing file {i+1}/{len(images)} : {image_file}")

    full_image_path = os.path.join(full_images_folder, image_file)
    raster_data = RasterData(full_image_path)

    sahi_result = sahi_model.predict(
        image=raster_data.as_rgb_img(),
    )

    sahi_predictions = sahi_model.get_prediction_data()

    sahi_predictions["geometry"] = (
        gpd.GeoSeries(
            data=[sg.Polygon(coords) for coords in sahi_predictions["bounding_box"]]
        )
        .affine_transform(raster_data.get_shapely_transform())
    )

    detections_gdf = gpd.GeoDataFrame(
        data=sahi_predictions,
        crs=RD_crs,
    )

    detections_gdf.insert(0, column="source_file", value=image_file)

    output_file = os.path.join(output_folder, f"{os.path.splitext(image_file)[0]}.geojson")
    detections_gdf.to_file(output_file, driver='GeoJSON')

    print(f"Predictions written to {output_file}")
    print("-----")

    detections_gdf_list.append(detections_gdf)

In [ ]:
full_detections_gdf = pd.concat(detections_gdf_list)
full_detections_gdf.reset_index(inplace=True, drop=True)
output_file = os.path.join(output_folder, "combined_detections.geojson")
full_detections_gdf.to_file(output_file, driver='GeoJSON', index=True)

In [ ]:
centrum = gpd.read_file("../datasets/experiments/parkeren/GBD_centrum_RD.json")
centrum_detections_gdf = full_detections_gdf[full_detections_gdf.intersects(centrum.geometry[0])]
output_file = os.path.join(output_folder, "combined_detections_centrum.geojson")
centrum_detections_gdf.to_file(output_file, driver='GeoJSON', index=True)

In [ ]:
# Read back from file
centrum_detections_gdf = gpd.read_file("../datasets/experiments/parkeren/detections/combined_detections_centrum.geojson").set_index("index")

## Detections EDA

In [ ]:
centrum_detections_gdf = gpd.read_file("../datasets/experiments/parkeren/detections/combined_detections_centrum.geojson").set_index("index")

In [ ]:
centrum_detections_gdf.area[centrum_detections_gdf.area <= 60].hist(bins=50)

In [ ]:
centrum_detections_gdf.area.describe(percentiles=[0.1, 0.2])

In [ ]:
ax = centrum_detections_gdf["confidence"].hist(bins=50, cumulative=True)
ax.set(xlabel="Confidence score", ylabel="Occurances")
ax.figure.savefig("../datasets/experiments/parkeren/histogram_conf_cumulative.png", dpi=150)

In [ ]:
centrum_detections_gdf["confidence"].describe(percentiles=[0.1, 0.2])

In [ ]:
def get_shape(geom):
    coords = geom.exterior.coords

    # get length of bounding box edges
    edge_length = (sg.Point(coords[0]).distance(sg.Point(coords[1])), sg.Point(coords[1]).distance(sg.Point(coords[2])))

    return {
        "length": max(edge_length),
        "width": min(edge_length)
    }

centrum_detections_gdf[["length", "width"]] = centrum_detections_gdf.apply(lambda row: get_shape(row["geometry"]), axis='columns', result_type='expand')

In [ ]:
ax = centrum_detections_gdf["width"].hist(bins=50)
ax.set(xlabel="Width", ylabel="Occurances")
ax.figure.savefig("../datasets/experiments/parkeren/histogram_width.png", dpi=150)

In [ ]:
ax = centrum_detections_gdf["length"].hist(bins=50)
ax.set(xlabel="Length", ylabel="Occurances")
ax.figure.savefig("../datasets/experiments/parkeren/histogram_length.png", dpi=150)

In [ ]:
car_dimensions = {
    "min_width": 1.4,
    "min_length": 3.0,
    "max_width": 3.0,
    "max_length": 12.0
}

In [ ]:
conf_treshold = 0.04
area_threshold = 5.0

def accept(row):
    accept_conf = row["confidence"] >= conf_treshold
    accept_width = (
        (row["width"] >= car_dimensions["min_width"])
        and (row["width"] <= car_dimensions["max_width"])
    )
    accept_length = (
        (row["length"] >= car_dimensions["min_length"])
        and (row["length"] <= car_dimensions["max_length"])
    )
    return accept_conf and accept_width and accept_length

centrum_detections_gdf["accept"] = centrum_detections_gdf.apply(accept, axis="columns")

In [ ]:
centrum_detections_gdf["accept"].value_counts()

## Crop and show detections

In [ ]:
full_image_path = f"../datasets/experiments/parkeren/luchtfotos/luchtfotos_centrum_2025/2025_121000_486000_RGB_JPEG_hrl.tif"

raster_data = RasterData(full_image_path)

x = 300
y = 600
w = 100
h = 100

cropped_img, cropped_poly = raster_data.get_relative_crop((x, y, x+w, y+h))

cropped_detections = centrum_detections_gdf[centrum_detections_gdf.intersects(cropped_poly)]

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np

padding = 10

fig, ax = plt.subplots(1, figsize=(10, 10), constrained_layout=True)

[x_min, y_min, x_max, y_max] = map(int, cropped_poly.bounds)

ax.imshow(cropped_img, extent=[x_min, x_max, y_min, y_max])

colors = [(0, 1, 0) if accept else (1, 0, 0) for accept in cropped_detections["accept"]]
cropped_detections.boundary.plot(ax=ax, color=colors)

ax.set_xlabel('X')
ax.set_ylabel('Y')

ax.set_xticks(range(x_min, x_max+1, 100))
ax.set_xticklabels(range(x_min, x_max+1, 100))
ax.set_yticks(range(y_min, y_max+1, 100))
ax.set_yticklabels(range(y_min, y_max+1, 100))

ax.set_xlim((x_min - padding, x_max + padding))
ax.set_ylim((y_min - padding, y_max + padding))
ax.set_aspect('equal', adjustable='box')

# extent = ax.get_window_extent().transformed(fig.dpi_scale_trans.inverted())
# plt.savefig("../datasets/experiments/parkeren/full_combined.jpg", bbox_inches=extent, dpi=450)

plt.show()